# Basic CTD Processing Pipeline Demo

This notebook will provide an example of basic pipeline functionality using CTD processing as the subject. It is extremely uncommon to collect glider data without CTD measurements so generally, this processing can be applied to almost any glider.

The data used here can be found at the [BOCD Bio-Carbon Deployment Catalogue](https://platforms.bodc.ac.uk/deployment-catalogue/BIO-Carbon/) under [Nelson (unit_397) OG NetCDF](https://linkedsystems.uk/erddap/files/Public_OG1_Data_001/Nelson_20240528/Nelson_646_R.nc). Context for the deployment can be found on the download site, but for further detail see the [NOC BIO-Carbon Project Page](https://noc.ac.uk/projects/bio-carbon).

Once you have downloaded the data, it should be placed in the **examples/data/OG1** folder or, alternatively, you can edit the config for this notebook (**examples/configs/example_config_nelson.yaml**) so that "file_path" in the "Load OG1" step points towards where your data is stored.


## Viewing the data

This isn't necessary for the pipeline to work, however it is useful to see that data for those who are unfamiliar with glider data

In [1]:
import xarray as xr

dataset = xr.load_dataset("../data/OG1/Nelson_646_R.nc")
dataset

<xarray.Dataset> Size: 443MB
Dimensions:                               (N_MEASUREMENTS: 2014856, N_PARAM: 24)
Dimensions without coordinates: N_MEASUREMENTS, N_PARAM
Data variables: (12/68)
    TIME                                  (N_MEASUREMENTS) datetime64[ns] 16MB ...
    TIME_GPS                              (N_MEASUREMENTS) datetime64[ns] 16MB ...
    PHASE                                 (N_MEASUREMENTS) float32 8MB nan .....
    PHASE_QC                              (N_MEASUREMENTS) float32 8MB nan .....
    CNDC                                  (N_MEASUREMENTS) float32 8MB 0.0 .....
    PRES                                  (N_MEASUREMENTS) float32 8MB 0.0 .....
    ...                                    ...
    PLATFORM_TYPE                         <U6 24B 'slocum'
    PLATFORM_MODEL                        <U2 8B 'G2'
    WMO_IDENTIFIER                        <U3 12B '830'
    DEPLOYMENT_TIME                       datetime64[ns] 8B 2024-05-27T23:00:00
    DEPLOYMENT_LATITUDE                   <U17 68B '50.89280319213867'
    DEPLOYMENT_LONGITUDE                  <U19 76B '-1.3955066204071045'
Attributes: (12/68)
    geospatial_bounds_crs:           EPSG:4326
    geospatial_bounds_vertical_crs:  EPSG:5831
    geospatial_lat_min:              50.890984
    geospatial_lat_max:              62.482586
    geospatial_lon_min:              50.890984
    geospatial_lon_max:              62.482586
    ...                              ...
    instrument:                      ['SBE Slocum Glider Payload (GPCTD) CTD'...
    metadata_link:                   https://api.linked-systems.uk/api/meta/v...
    trajectory:                      Nelson_20240528
    date_created:                    2024-09-30T08:14:20.465997
    date_modified:                   2024-09-30T08:14:20.466021
    id:                              Nelson_20240528T000000_R

Feel free to explore the data variables in this dataset. Many will be completely filled with Nans, but for our purposes we only care about TIME, LATITUDE, LONGITUDE, PRES (pressure), CNDC (conductivity), and TEMP (temperature). You may notice that this data has not coordinates, which means it does not conform to the glider-community [OG1 format](https://github.com/OceanGlidersCommunity/OG-format-user-manual/blob/main/OG_Format.adoc) however, it is still compatable with the pipeline so long as the N_MEASUREMENTS dimension exists.

## Importing the pipeline

Before anything can be run, we need to make sure that the necessary packages are installed. All of the requirements are listed in **requirements.txt** and can be installed automatically using `pip install -r requirements.txt` in a terminal (if using anaconda, make sure that you have selected the right environment using `conda activate your_env_name`). If you are missing any packages the pipeline may break.

When we import the pipeline it will try and register all of the steps available to it. This is labeled as [Discovery] in the print log. If you are missing a package requirement, you will see a print in the form "Failed to import <step>: <error message>" which should indicate which package you are missing.

In [2]:
# Set the working directory and import the pipeline
import os
cwd = os.getcwd()
os.chdir(f"{cwd}/../../src")
from toolbox.pipeline import *

[Discovery] Scanning for step modules in C:\Users\banga\OneDrive - University of Southampton\Documents\PhD\RISC\toolbox\src\toolbox\steps\custom
[Discovery] Importing step module: toolbox.steps.custom.apply_qc
[Discovery] Importing step module: toolbox.steps.custom.blank_step
[Discovery] Importing step module: toolbox.steps.custom.derive_ctd
[Discovery] Importing step module: toolbox.steps.custom.explore_variables
[Discovery] Failed to import toolbox.steps.custom.explore_variables: name 'QCHandlingMixin' is not defined
[Discovery] Importing step module: toolbox.steps.custom.export
[Discovery] Importing step module: toolbox.steps.custom.find_profiles
[Discovery] Importing step module: toolbox.steps.custom.gen_data
[Discovery] Importing step module: toolbox.steps.custom.interpolate_data
[Discovery] Importing step module: toolbox.steps.custom.load_data
[Discovery] Importing step module: toolbox.steps.custom.profile_direction
[Discovery] Importing step module: toolbox.steps.custom.qc.archi

## Making the pipeline

To make the pipeline, we simply have to make an instance using the `Pipeline()` class, passing it the path to the config which defines how we want to process the data. You should see a series of print logs indicating that the steps specified in the config have been found and added to the pipeline.

In [3]:
# Create the pipeline using the specified config
Pipe = Pipeline(
    config_path=r"../examples/configs/example_config_nelson.yaml"
)

Step 'Load OG1' added successfully!
Step 'Apply QC' added successfully!
Step 'Apply QC' added successfully!
Step 'Interpolate Data' added successfully!
Step 'Derive CTD' added successfully!
Step 'Find Profiles' added successfully!
Step 'Apply QC' added successfully!
Step 'Find Profile Direction' added successfully!
Step 'Salinity Adjustment' added successfully!
Step 'Derive CTD' added successfully!
Step 'Data Export' added successfully!


## Running the pipeline

Calling the Pipeline.run() method tells the pipeline to execute all of the steps listed in the config in order from top-down. As it is running, it will plot diagnostics for steps where the setting is True. For more details on what is being run, see the comments in the config. Once completed, the processing has been excecuted and your data should be saved as **Nelson_646_R_Processed.nc** in the data/OG1 folder.

In [9]:
# Run the pipeline
Pipe.run()

Executing: Load OG1
Executing: Apply QC
[Apply QC] Data found in context.
[Apply QC] Applying: impossible date test
[Apply QC] Applying: impossible location test
[Apply QC] Applying: position on land test
[Apply QC] Applying: impossible speed test
Executing: Apply QC
[Apply QC] Data found in context.
[Apply QC] Applying: range test
[Apply QC] Applying: stuck value test
[Interpolate Data] Data found in context.
Executing: Interpolate Data
[Interpolate Data] Interpolating variables...
[Derive CTD] Data found in context.
Executing: Derive CTD
[Derive CTD] Processing CTD...
[Derive CTD] Deriving DEPTH...
[Find Profiles] Data found in context.
Executing: Find Profiles
[Find Profiles] Attempting to designate profile numbers
[Find Profiles] Generating diagnostics
[Find Profiles] continuing with parameters: 
  Gradient Thresholds: [0.033, -0.033]
  Filter Window Sizes: ['20s', '20s']
  Depth column: DEPTH

Executing: Apply QC
[Apply QC] Data found in context.
[Apply QC] Applying: valid profile

Progress: 100%|██████████| 161/161 [00:31<00:00,  5.17prof/s]


[Derive CTD] Data found in context.
Executing: Derive CTD
[Derive CTD] Processing CTD...
[Derive CTD] Deriving PRAC_SALINITY...
[Derive CTD] Deriving ABS_SALINITY...
[Derive CTD] Deriving CONS_TEMP...
[Derive CTD] Deriving DENSITY...
Executing: Data Export
[Data Export] Exporting data in netcdf format to ../examples/data/OG1/Nelson_646_R_Processed.nc
[Data Export] Data found in context.
[Data Export] Data exported successfully to ../examples/data/OG1/Nelson_646_R_Processed.nc


# Checking the output

Because our pipeline "Pipe" is stored in pythons local variable space we can actually access the data without having to load in the processed data as we did for the input data. This can be done by looking in the pipeline context as follows:

In [5]:
data = Pipe._context["data"]
data

<xarray.Dataset> Size: 887MB
Dimensions:                                  (N_MEASUREMENTS: 2014856,
                                              N_PARAM: 24)
Dimensions without coordinates: N_MEASUREMENTS, N_PARAM
Data variables: (12/98)
    TIME                                     (N_MEASUREMENTS) datetime64[ns] 16MB ...
    TIME_GPS                                 (N_MEASUREMENTS) datetime64[ns] 16MB ...
    PHASE                                    (N_MEASUREMENTS) float32 8MB nan...
    PHASE_QC                                 (N_MEASUREMENTS) int64 16MB 9 ... 9
    CNDC                                     (N_MEASUREMENTS) float32 8MB 0.0...
    PRES                                     (N_MEASUREMENTS) float32 8MB 0.0...
    ...                                       ...
    ABS_SALINITY                             (N_MEASUREMENTS) float64 16MB na...
    ABS_SALINITY_QC                          (N_MEASUREMENTS) int64 16MB 9 ... 9
    CONS_TEMP                                (N_MEASUREMENTS) float64 16MB na...
    CONS_TEMP_QC                             (N_MEASUREMENTS) int64 16MB 9 ... 9
    DENSITY                                  (N_MEASUREMENTS) float64 16MB na...
    DENSITY_QC                               (N_MEASUREMENTS) int64 16MB 9 ... 9
Attributes: (12/68)
    geospatial_bounds_crs:           EPSG:4326
    geospatial_bounds_vertical_crs:  EPSG:5831
    geospatial_lat_min:              50.890984
    geospatial_lat_max:              62.482586
    geospatial_lon_min:              50.890984
    geospatial_lon_max:              62.482586
    ...                              ...
    instrument:                      ['SBE Slocum Glider Payload (GPCTD) CTD'...
    metadata_link:                   https://api.linked-systems.uk/api/meta/v...
    trajectory:                      Nelson_20240528
    date_created:                    2024-09-30T08:14:20.465997
    date_modified:                   2024-09-30T08:14:20.466021
    id:                              Nelson_20240528T000000_R

If you compare the number of data variables here to that of the input data, we have gained 30 variables - these will be a mixture of our derived variables and new QC columns.

I'd recommend checking the data with some plotting to make sure everything looks ok - in fact, with this dataset not everything is ok with the input CTD variables. If you look at the salinity outputs, they are much smaller than expected. This is because the CNDC input to the pipline was in the wrong units for the gsw-python implementation of the equation of state of seawater. This kind of error would have to be rectified manually by modifying the input data as the pipeline expects correct unit inputs.